# Reddit Search Example

Just a simple example of how to search something on reddit and get some posts and comments

In [2]:
import praw

RESULT_LIMIT = 10

r = praw.Reddit(user_agent='sample_app')
nba_subreddit = r.get_subreddit('nba')

stephcurry_search = nba_subreddit.search('Stephen Curry')

In [67]:
post = stephcurry_search.next()

print post
print post.url
print post.ups
print post.num_comments


256 :: [Question] What did Stephen Curry do that contributed to him being les...
https://www.reddit.com/r/nba/comments/3iyowa/question_what_did_stephen_curry_do_that/
256
232


In [4]:
comments = []

for i in post.comments:
    try:
        comment_str = str(i.body.encode('utf-8'))
        comments.append(comment_str)
    except AttributeError:
        print "End of List found"
        if len(comments) != 0:
            print "comments list is filled"

End of List found
comments list is filled


In [40]:
# Analyze the comments as positive or negative

# build a generic analyzer from an example 
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
print unigram_feats
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
#for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    #print('{0}: {1}'.format(key, value))


[u'.', u'the', u',', u'a', u'and', u'of', u'to', u'is', u'in', u'with', u'it', u'that', u'his', u'for', u'on', u'an', u'who', u'by', u'he', u'her', u'"', u'from', u'as', u'film', u'movie', u'this', u'their', u'but', u'one', u'at', u'the_NEG', u'about', u'a_NEG', u"there's", u'to_NEG', u'story', u'are', u'(', u'when', u'so', u'they', u',_NEG', u'be', u')', u'life', u'not', u'you', u'all', u'what', u'into', u'out', u'have', u'she', u'will', u'like', u'even', u'has', u'can', u'only', u'--', u'more', u'its', u':', u';', u'if', u'where', u'search', u'most', u'him', u'look', u"it's", u'home', u'them', u'begins', u'make', u'love', u'but_NEG', u'of_NEG', u'two', u'both', u'some', u'which', u'made']
Training classifier


In [43]:
phrase = [u"you", u"suck", u"did", u"you", u"know", u"that"]
print sentim_analyzer.classify((phrase))
print(classifier.classify(word_feats(['magnificent'])))


subj
subj


In [44]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords

def word_feats(words):
    return dict([(word, True) for word in words])

negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4

trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))

classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

train on 1500 instances, test on 500 instances
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [77]:
#print(classifier.classify(word_feats(['didnt', 'you', 'really', 'think', 'that', 'was', 'great', '?'])))
for i in range(0,len(comments)):    
    for comment in comments[i].split('\n'):
        if any(i in str(comment) for i in 'Stephen Curry'.lower().split(' ')):
            print comment
            print(classifier.classify(word_feats(comment)))


curry
neg
Curry and Harden are terrible, terrible defenders, and I'm pretty sure Curry is the worse of the two. Everyone says Curry is in line to be the best PG in the league but I don't think he'll ever be more than top 3 without any defensive skills. Harden has more potential than curry imo but I have a bad feeling he won't reach it. 
neg
Stephen curry is not at the level of george or harden. 
pos


In [78]:
print dir(post)

['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattr__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_api_link', '_comment_sort', '_comments', '_comments_by_id', '_extract_more_comments', '_get_json_dict', '_has_fetched', '_info_url', '_insert_comment', '_methods', '_orphaned', '_params', '_populate', '_post_populate', '_replaced_more', '_underscore_names', '_uniq', '_update_comments', 'add_comment', 'approve', 'approved_by', 'archived', 'author', 'author_flair_css_class', 'author_flair_text', 'banned_by', 'clear_vote', 'clicked', 'comments', 'created', 'created_utc', 'delete', 'distinguish', 'distinguished', 'domain', 'downs', 'downvote', 'edit', 'edited', 'from', 'from_api_response', 'from_id', 'from_json', 'from_kind', 'from_url', 'fullname', 'get_duplicates', 'get_fl